In [1]:
from definitions import path_join, VENSIM_MODELS_DIR, make_directory, DATA_DIR

model_name = 'teacup'
vensim_model_file = path_join(VENSIM_MODELS_DIR, '{}.mdl'.format(model_name))

dataset_dir = path_join(DATA_DIR, model_name)
make_directory(dataset_dir)

dataset_file = path_join(dataset_dir, 'dataset.csv')

In [2]:
import pysd

In [3]:
model = pysd.read_vensim(vensim_model_file)

In [4]:
model.doc()

,Real Name,Py Name,Unit,Type,Comment
0,Characteristic Time,characteristic_time,"Minutes [0,?]",constant,How long will it take the teacup to cool 1/e o...
1,FINAL TIME,final_time,Minute,constant,The final time for the simulation.
2,Heat Loss to Room,heat_loss_to_room,Degrees Fahrenheit/Minute,component,This is the rate at which heat flows from the ...
3,INITIAL TIME,initial_time,Minute,constant,The initial time for the simulation.
4,Room Temperature,room_temperature,"Degrees Fahrenheit [-459.67,?]",constant,Put in a check to ensure the room temperature ...
5,SAVEPER,saveper,"Minute [0,?]",component,The frequency with which output is stored.
6,TIME STEP,time_step,"Minute [0,?]",constant,The time step for the simulation.
7,Teacup Temperature,teacup_temperature,"Degrees Fahrenheit [32,212]",component,The model is only valid for the liquid phase o...


In [5]:
stocks = model.run()

In [6]:
stocks.iloc[:10]

,Characteristic Time,FINAL TIME,Heat Loss to Room,INITIAL TIME,Room Temperature,SAVEPER,TIME,TIME STEP,Teacup Temperature,Time
0.000,10,30,11.000000,0,70,0.125,0.000,0.125,180.000000,0.000
0.125,10,30,10.862500,0,70,0.125,0.125,0.125,178.625000,0.125
0.250,10,30,10.726719,0,70,0.125,0.250,0.125,177.267188,0.250
0.375,10,30,10.592635,0,70,0.125,0.375,0.125,175.926348,0.375
0.500,10,30,10.460227,0,70,0.125,0.500,0.125,174.602268,0.500
0.625,10,30,10.329474,0,70,0.125,0.625,0.125,173.294740,0.625
0.750,10,30,10.200356,0,70,0.125,0.750,0.125,172.003556,0.750
0.875,10,30,10.072851,0,70,0.125,0.875,0.125,170.728511,0.875
1.000,10,30,9.946940,0,70,0.125,1.000,0.125,169.469405,1.000
1.125,10,30,9.822604,0,70,0.125,1.125,0.125,168.226037,1.125


In [7]:
iters_count = 50
final_time = iters_count * 0.125

parameters_grid = {
    'FINAL TIME': [final_time],
    'Teacup Temperature': [i * 10 for i in range(1, 20)],
    'Room Temperature': [i * 5 for i in range(1, 20)],
#     'Characteristic Time': [i for i in range(1, 20)],
}

In [8]:
from sklearn.model_selection import ParameterSampler
seed = 123
sampler = ParameterSampler(parameters_grid, n_iter=300, random_state=seed)

In [9]:
parts = []

for _index, _params in enumerate(sampler):
    data_part = model.run(
        params={
            'FINAL TIME': _params['FINAL TIME'],
            'Room Temperature': _params['Room Temperature'],
#             'Characteristic Time': _params['Characteristic Time'],
        },
        initial_condition=(0, {'Teacup Temperature': _params['Teacup Temperature'],}),
    )
    data_part['sim_index'] = _index
    parts.append(data_part)

In [10]:
import pandas as pd

dataset = pd.concat(parts, ignore_index=True)

In [11]:
dataset.shape

(15300, 11)

In [12]:
dataset.columns

Index(['Characteristic Time', 'FINAL TIME', 'Heat Loss to Room',
       'INITIAL TIME', 'Room Temperature', 'SAVEPER', 'TIME', 'TIME STEP',
       'Teacup Temperature', 'Time', 'sim_index'],
      dtype='object')

In [13]:
columns = ['sim_index', 'Characteristic Time', 'Room Temperature', 'Teacup Temperature']
final_dataset = dataset[columns]

In [14]:
final_dataset.shape

(15300, 4)

In [15]:
final_dataset.describe()

,sim_index,Characteristic Time,Room Temperature,Teacup Temperature
count,15300.00000,15300.0,15300.0000,15300.000000
mean,149.50000,10.0,49.9000,86.815004
std,86.60489,0.0,27.3503,42.356314
min,0.00000,10.0,5.0000,7.665786
25%,74.75000,10.0,25.0000,51.001572
50%,149.50000,10.0,50.0000,86.350404
75%,224.25000,10.0,75.0000,119.317563
max,299.00000,10.0,95.0000,190.000000


In [16]:
final_dataset[final_dataset['sim_index'] == 0]

,sim_index,Characteristic Time,Room Temperature,Teacup Temperature
0,0,10,45,140.000000
1,0,10,45,138.812500
2,0,10,45,137.639844
3,0,10,45,136.481846
4,0,10,45,135.338323
5,0,10,45,134.209094
6,0,10,45,133.093980
7,0,10,45,131.992805
8,0,10,45,130.905395
9,0,10,45,129.831578


In [17]:
final_dataset.head()

,sim_index,Characteristic Time,Room Temperature,Teacup Temperature
0,0,10,45,140.000000
1,0,10,45,138.812500
2,0,10,45,137.639844
3,0,10,45,136.481846
4,0,10,45,135.338323


In [18]:
final_dataset = final_dataset.round(2)

In [19]:
final_dataset.head()

,sim_index,Characteristic Time,Room Temperature,Teacup Temperature
0,0,10,45,140.00
1,0,10,45,138.81
2,0,10,45,137.64
3,0,10,45,136.48
4,0,10,45,135.34


In [20]:
final_dataset.to_csv(dataset_file, index=False)

In [21]:
_readed = pd.read_csv(dataset_file)

In [22]:
_readed.head()

,sim_index,Characteristic Time,Room Temperature,Teacup Temperature
0,0,10,45,140.00
1,0,10,45,138.81
2,0,10,45,137.64
3,0,10,45,136.48
4,0,10,45,135.34
